In [1]:
import functools

import numpy as np
from brainio.stimuli import StimulusSet
import brainscore_vision
from brainio.assemblies import DataAssembly, BehavioralAssembly, walk_coords
from brainscore_vision.benchmark_helpers.screen import place_on_screen
from brainscore_vision.model_helpers.activations import PytorchWrapper
from brainscore_vision.model_helpers.brain_transformation import ModelCommitment

/Users/linussommer/miniforge3/envs/bs_old/lib/python3.7/site-packages/brainscore_core/metrics/__init__.py:16: FutureWarning: xarray subclass Score should explicitly define __slots__
  class Score(DataAssembly):


In [2]:
def pytorch_custom():
    import torch
    from torch import nn
    from brainscore_vision.model_helpers.activations.pytorch import load_preprocess_images

    class MyModel(nn.Module):
        def __init__(self):
            super(MyModel, self).__init__()
            np.random.seed(0)
            torch.random.manual_seed(0)
            self.conv1 = torch.nn.Conv2d(in_channels=3, out_channels=2, kernel_size=3)
            self.relu1 = torch.nn.ReLU()
            linear_input_size = np.power((224 - 3 + 2 * 0) / 1 + 1, 2) * 2
            self.linear = torch.nn.Linear(int(linear_input_size), 1000)
            self.relu2 = torch.nn.ReLU()

        def forward(self, x):
            x = self.conv1(x)
            x = self.relu1(x)
            x = x.view(x.size(0), -1)
            x = self.linear(x)
            x = self.relu2(x)
            return x

    preprocessing = functools.partial(load_preprocess_images, image_size=224)
    return PytorchWrapper(model=MyModel(), preprocessing=preprocessing)

In [3]:
def calculate_similarity_matrix(features, similarity_measure='dot'):
   print(features, type(features))
   features = features.transpose('presentation', 'neuroid')
   values = features.values
   if similarity_measure == 'dot':
      similarity_matrix = np.dot(values, np.transpose(values))
   elif similarity_measure == 'cosine':
      row_norms = np.linalg.norm(values, axis=1).reshape(-1, 1)
      norm_product = np.dot(row_norms, row_norms.T)
      dot_product = np.dot(values, np.transpose(values))
      similarity_matrix = dot_product / norm_product
   else:
      raise ValueError(
      f"Unknown similarity_measure {similarity_measure} -- expected one of 'dot' or 'cosine'")

   similarity_matrix = DataAssembly(similarity_matrix, coords={
        **{f"{coord}_left": ('presentation_left', values) for coord, _, values in
           walk_coords(features['presentation'])},
        **{f"{coord}_right": ('presentation_right', values) for coord, _, values in
           walk_coords(features['presentation'])}
   }, dims=['presentation_left', 'presentation_right'])
   return similarity_matrix

In [4]:
def calculate_choices(similarity_matrix, triplets):
    triplets = np.array(triplets).reshape(-1, 3)
    choice_predictions = []
    for triplet in triplets:
        i, j, k = triplet
        sims = similarity_matrix[i, j], similarity_matrix[i, k],  similarity_matrix[j, k]
        idx = triplet[2 - np.argmax(sims)]
        choice_predictions.append(idx)
    # TODO return as DataAssembly
    return choice_predictions

In [5]:
import numpy as np
import pandas as pd
from brainscore_vision import load_stimulus_set, load_dataset 

assembly = load_dataset('Hebart2023')
stimulus_set = load_stimulus_set("Hebart2023")
flat_triplets = np.array([
    assembly.coords["image_1"].values,
    assembly.coords["image_2"].values,
    assembly.coords["image_3"].values
]).T.reshape(-1, 1)

In [6]:
ss = pd.DataFrame(columns=stimulus_set.columns)
image_paths = []
for stim in flat_triplets[:99]:
    ss = ss.append(stimulus_set.loc[stim])
    image_paths.append(stimulus_set.stimulus_paths[stim[0]])

ss = StimulusSet(ss)
ss.identifier = 'Hebart2023'
ss.stimulus_paths = image_paths

print(ss)

     stimulus_id          top_down_1     rank          Wordnet_ID4  \
112          112                 NaN  18754.0         beehive.n.02   
1459        1459              animal  15107.0           skunk.n.04   
632          632                 NaN   8561.0       footprint.n.01   
1278        1278                 NaN      NaN      rain_gauge.n.01   
1561        1561           furniture   6068.0           stool.n.01   
...          ...                 ...      ...                  ...   
1451        1451                 NaN  13116.0          skewer.n.01   
1635        1635                 toy      NaN           teddy.n.01   
1335        1335                 NaN      NaN  roulette_wheel.n.01   
250          250           container   2967.0             can.n.01   
1764        1764  musical instrument   6761.0          violin.n.01   

           unique_id              example_image          top_down_2  filename  \
112          beehive  https://imgur.com/blRjwaL                 NaN   112.jpg 

In [ ]:
# create model
activations_model = pytorch_custom()
layers = ["relu2"]

# create brain model
brain_model = ModelCommitment(
    identifier=activations_model.identifier, 
    activations_model=activations_model, 
    layers=[None], 
    behavioral_readout_layer='relu2')

# get activations
stimuli = place_on_screen(
    stimulus_set=ss,
    target_visual_degrees=brain_model.visual_degrees(),
    source_visual_degrees=8)

In [ ]:
unique_stimuli = ss.drop_duplicates(subset=['stimulus_id'])
features = activations_model(unique_stimuli, layers=layers)
features = features.transpose('presentation', 'neuroid')

In [ ]:
features

<xarray.NeuroidAssembly (presentation: 1854, neuroid: 1000)>
array([[0.        , 0.04621074, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.00653093, ..., 0.        , 0.07874878,
        0.02063107],
       [0.        , 0.07153837, 0.        , ..., 0.        , 0.        ,
        0.24205582],
       ...,
       [0.08684272, 0.21608943, 0.        , ..., 0.37696987, 0.        ,
        0.20030946],
       [0.        , 0.43397385, 0.        , ..., 0.11165395, 0.04454674,
        0.12716481],
       [0.        , 0.35502172, 0.        , ..., 0.32331127, 0.        ,
        0.24606182]], dtype=float32)
Coordinates:
  * neuroid           (neuroid) MultiIndex
  - neuroid_num       (neuroid) int64 0 1 2 3 4 5 6 ... 994 995 996 997 998 999
  - model             (neuroid) object 'MyModel' 'MyModel' ... 'MyModel'
  - layer             (neuroid) object 'relu2' 'relu2' ... 'relu2' 'relu2'
  - channel           (neuroid) int64 0 1 2 3 4 5 6 ... 994 995 996 997 998 999
  - channel_x         (neuroid) float64 nan nan nan nan nan ... nan nan nan nan
  - channel_y         (neuroid) float64 nan nan nan nan nan ... nan nan nan nan
  - neuroid_id        (neuroid) object 'MyModel.relu2.0' ... 'MyModel.relu2.999'
    top_down_1        (presentation) object nan 'animal' nan nan ... nan nan nan
    rank              (presentation) float64 1.875e+04 1.511e+04 ... 2.933e+04
    Wordnet_ID4       (presentation) object 'beehive.n.02' ... 'sunroof.n.01'
    unique_id         (presentation) object 'beehive' 'skunk' ... 'sunroof'
    example_image     (presentation) object 'https://imgur.com/blRjwaL' ... '...
    top_down_2        (presentation) object nan 'animal' ... nan 'part of car'
    filename          (presentation) object '112.jpg' '1459.jpg' ... '1586.jpg'
    Wordnet_ID2       (presentation) object 'beehive%1:17:00::' ... 'sunroof%...
    dispersion        (presentation) float64 0.9 0.9 0.93 nan ... 0.91 nan 0.87
    bottom_up         (presentation) object nan 'animal' ... nan 'part of car'
    word_freq         (presentation) float64 467.0 738.0 2.022e+03 ... nan 160.0
    dominant_part     (presentation) object 'Noun' 'Noun' 'Noun' ... nan 'Noun'
    freq_1            (presentation) float64 78.0 42.0 146.0 ... 74.0 nan 57.0
    WordNet_synonyms  (presentation) object 'beehive, hive' ... 'sunroof, sun...
    freq_2            (presentation) float64 44.0 166.0 55.0 ... 92.0 0.0 22.0
    WordNet_ID        (presentation) object 'n09218159' ... 'n04357121'
    Wordnet_ID3       (presentation) object 'beehive#2' ... 'sunroof#1'
    word_freq_online  (presentation) object 537 806 1606 31 ... 891 849 5 220
  * presentation      (presentation) MultiIndex
  - stimulus_id       (presentation) object 112 1459 632 1278 ... 451 508 1586

In [ ]:
sim = calculate_similarity_matrix(features, similarity_measure='dot')
print(sim.shape)

<xarray.NeuroidAssembly (presentation: 1854, neuroid: 1000)>
array([[0.        , 0.04621074, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.00653093, ..., 0.        , 0.07874878,
        0.02063107],
       [0.        , 0.07153837, 0.        , ..., 0.        , 0.        ,
        0.24205582],
       ...,
       [0.08684272, 0.21608943, 0.        , ..., 0.37696987, 0.        ,
        0.20030946],
       [0.        , 0.43397385, 0.        , ..., 0.11165395, 0.04454674,
        0.12716481],
       [0.        , 0.35502172, 0.        , ..., 0.32331127, 0.        ,
        0.24606182]], dtype=float32)
Coordinates:
  * neuroid           (neuroid) MultiIndex
  - neuroid_num       (neuroid) int64 0 1 2 3 4 5 6 ... 994 995 996 997 998 999
  - model             (neuroid) object 'MyModel' 'MyModel' ... 'MyModel'
  - layer             (neuroid) object 'relu2' 'relu2' ... 'relu2' 'relu2'
  - channel           (neuroid) int64 0 1 2 3 4 5 6 ... 994 995

In [72]:
trips = flat_triplets[:33]
print(trips)

[[ 112]
 [1459]
 [ 632]
 [1278]
 [1561]
 [ 792]
 [ 796]
 [1660]
 [ 413]
 [  27]
 [ 671]
 [ 320]
 [1448]
 [1531]
 [ 324]
 [1594]
 [ 280]
 [1825]
 [1381]
 [1522]
 [  14]
 [1528]
 [ 526]
 [ 208]
 [1491]
 [1336]
 [1850]
 [1494]
 [ 171]
 [1113]
 [ 659]
 [1332]
 [1605]]


In [73]:
choices = calculate_choices(sim, trips)
print(choices)

[1459, 1561, 796, 27, 324, 280, 1381, 1528, 1850, 171, 1605]


In [ ]:
from brainscore_vision import score

similarity_score = score(model_identifier='alexnet', benchmark_identifier='Hebart2023')
similarity_score